# Задание 4.

Загрузите [данные по изменению температуры поверхности земли](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data). Для этого может понадобится зарегистрироваться на [Kaggle](https://kaggle.com). Затем нужно будет работать с данными, которые содержатся в файле **GlobalLandTemperaturesByMajorCity.csv**

**NB** Все подсчеты необходимо делать с помощью `PySpark`, без применения `pandas api`. Можно использоать `SQL`.

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

schema = StructType([
    StructField("date", DateType(), True),
    StructField("AverageTemperature", FloatType(), True),
    StructField("AverageTemperatureUncertainty", FloatType(), True),
    StructField("City", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("Latitude", StringType(), True),
    StructField("Longitude", StringType(), True)])


data = spark\
    .read\
    .option("delimiter", ",")\
    .option("header","true")\
    .csv('./GlobalLandTemperaturesByCity.csv', schema=schema)


data.take(10)

[Row(date=datetime.date(1743, 11, 1), AverageTemperature=6.067999839782715, AverageTemperatureUncertainty=1.7369999885559082, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1743, 12, 1), AverageTemperature=None, AverageTemperatureUncertainty=None, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1744, 1, 1), AverageTemperature=None, AverageTemperatureUncertainty=None, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1744, 2, 1), AverageTemperature=None, AverageTemperatureUncertainty=None, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1744, 3, 1), AverageTemperature=None, AverageTemperatureUncertainty=None, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1744, 4, 1), AverageTemperature=5.788000106811523, AverageTemperatureUncertainty=3.624000072

# Задание 4.1 (1 балл)

В последующих заданиях будут учитываться данные начиная с 01.01.1950. Для этого создайте новый `DataFrame`, в котором удалены все строки до 01.01.1950. Используйте созданный DataFrame в последующих заданиях.  

In [4]:
from pyspark.sql.functions import *

data = data.where(data.date >= lit("1950-01-01"))

data.take(5)

[Row(date=datetime.date(1950, 1, 1), AverageTemperature=-0.7120000123977661, AverageTemperatureUncertainty=0.2800000011920929, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1950, 2, 1), AverageTemperature=1.4229999780654907, AverageTemperatureUncertainty=0.21799999475479126, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1950, 3, 1), AverageTemperature=4.165999889373779, AverageTemperatureUncertainty=0.26600000262260437, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1950, 4, 1), AverageTemperature=6.743000030517578, AverageTemperatureUncertainty=0.3100000023841858, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E'),
 Row(date=datetime.date(1950, 5, 1), AverageTemperature=12.71399974822998, AverageTemperatureUncertainty=0.2460000067949295, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E')]

# Задание 4.2 (2 балла)

Найдите город, для которого выборочная дисперсия температур на приведенных данных максимальна. 

In [5]:
max_unc = data.select(max(data.AverageTemperatureUncertainty).alias('max')).take(1)[0]['max']
max_unc

5.181000232696533

In [6]:
max_unc_row = data.filter(data.AverageTemperatureUncertainty == max_unc).take(1)
max_unc_row

[Row(date=datetime.date(2012, 12, 1), AverageTemperature=-22.714000701904297, AverageTemperatureUncertainty=5.181000232696533, City='Seversk', Country='Russia', Latitude='57.05N', Longitude='84.10E')]

In [7]:
max_unc_row[0]['City']

'Seversk'

# Задание 4.3 (2 баллов)

Посчитайте данные по среднегодовой температуре в Санкт-Петербурге. Определите года, в которых средняя температура была выше, чем в предыдущем  и последующем году. 

In [8]:
spb = data.filter(data.City == 'Saint Petersburg')
spb.take(3)

[Row(date=datetime.date(1950, 1, 1), AverageTemperature=-13.053000450134277, AverageTemperatureUncertainty=0.6650000214576721, City='Saint Petersburg', Country='Russia', Latitude='60.27N', Longitude='29.19E'),
 Row(date=datetime.date(1950, 2, 1), AverageTemperature=-5.828999996185303, AverageTemperatureUncertainty=0.2549999952316284, City='Saint Petersburg', Country='Russia', Latitude='60.27N', Longitude='29.19E'),
 Row(date=datetime.date(1950, 3, 1), AverageTemperature=-2.812000036239624, AverageTemperatureUncertainty=0.39800000190734863, City='Saint Petersburg', Country='Russia', Latitude='60.27N', Longitude='29.19E')]

In [15]:
spb = spb.withColumn('year', year(spb['date']))
spb.take(2)

[Row(date=datetime.date(1950, 1, 1), AverageTemperature=-13.053000450134277, AverageTemperatureUncertainty=0.6650000214576721, City='Saint Petersburg', Country='Russia', Latitude='60.27N', Longitude='29.19E', year=1950),
 Row(date=datetime.date(1950, 2, 1), AverageTemperature=-5.828999996185303, AverageTemperatureUncertainty=0.2549999952316284, City='Saint Petersburg', Country='Russia', Latitude='60.27N', Longitude='29.19E', year=1950)]

In [38]:
from pyspark.sql.window import Window

windowSpec  = Window.orderBy("year")

spb.groupBy('year')\
    .agg(mean('AverageTemperature').alias('mean_temp'))\
    .withColumn('next_year', lead('mean_temp').over(windowSpec)) \
    .withColumn('prev_year', lag('mean_temp').over(windowSpec)) \
    .dropna() \
    .filter((col('mean_temp') > col('next_year')) & (col('mean_temp') > col('prev_year'))) \
    .show()


+----+------------------+------------------+------------------+
|year|         mean_temp|         next_year|         prev_year|
+----+------------------+------------------+------------------+
|1953|13.636541654666265|13.202666802021364|12.903041779994965|
|1957| 13.73249996205171|12.398666789134344|12.292041632036367|
|1959|13.694666539629301|12.905833464115858|12.398666789134344|
|1961|14.023750064273676|13.002583354711533|12.905833464115858|
|1964|13.280166772504648|12.992083241542181|12.564458345373472|
|1967|13.436208200951418|12.361124955117702|12.236166739215454|
|1972|13.954541698098183| 13.31808334092299|13.243916774789492|
|1975|  14.4162916050603| 12.06716671337684|14.130666647106409|
|1977|12.816458304723104| 12.33083330343167| 12.06716671337684|
|1979|13.079250040153662|12.851041793823242| 12.33083330343167|
|1982|13.757541552186012|13.488666544357935|  13.0456249093016|
|1984|  13.4956249371171|12.624125062177578|13.488666544357935|
|1986|13.421125123898188|12.140208333730

# Задание 4.4 (4 балла)

Найдите города, для которых: 
1. Разница между максимальным и минимальным значением среднегодовой температуры в выборке максимальна.
2. Самая большая средняя разница между средней температурой января и средней температурой июля.
3. Наибольшее среднее количество месяцев с отрицательной температурой в году.

In [51]:
# 1. Разница между максимальным и минимальным значением среднегодовой температуры в выборке максимальна.

data.withColumn('year', year('date')) \
    .groupBy('year', 'City') \
    .agg(mean('AverageTemperature').alias('mean_temp')) \
    .groupBy('City')\
    .agg(max('mean_temp').alias('max_temp'),
         min('mean_temp').alias('min_temp')) \
    .withColumn('diff', col('max_temp') - col('min_temp')).sort(col('diff').desc()).take(1)

[Row(City='Norilsk', max_temp=-7.456666747728984, min_temp=-14.705333267649015, diff=7.2486665199200315)]

In [65]:
# 2. Самая большая средняя разница между средней температурой января и средней температурой июля.
mean_months = data.withColumn('month', month('date')) \
    .groupBy('City', 'month')\
    .agg(mean('AverageTemperature').alias('mean_temp')) \

january = mean_months.filter(col('month')==1) \
    .withColumnRenamed('mean_temp', 'jan_temp').select('City', 'jan_temp')

july = mean_months.filter(col('month')==7)\
    .withColumnRenamed('mean_temp', 'jul_temp').select('City', 'jul_temp')

july.join(january, july.City == january.City, 'inner')\
    .withColumn('diff', abs(col('jan_temp') - col('jul_temp'))).sort(col('diff').desc()).take(1)

[Row(City='Blagoveshchensk', jul_temp=21.186390727758408, City='Blagoveshchensk', jan_temp=-24.53853130340576, diff=45.72492203116417)]

In [64]:
# 3. Наибольшее среднее количество месяцев с отрицательной температурой в году.

data.filter(col('AverageTemperature') < 0)\
    .withColumn('year', year(col('date'))) \
    .groupBy('City', 'year') \
    .agg(count('AverageTemperature').alias('neg_months')) \
    .groupBy('City') \
    .agg(mean('neg_months').alias('mean_neg_months')) \
    .sort(col('mean_neg_months').desc()).take(1)

[Row(City='Norilsk', mean_neg_months=8.0625)]